In [15]:
from aerolib import *

In [2]:
#FIXME: перенести значения в файл настроек
Re = 250000
M = 0.0
ncr = 12
it = 1000
alpha_min = -3
alpha_max = 13
alpha_step = 0.2

In [3]:
def isNaN(num):
    return num != num

In [5]:
class InsufficientInputData(Exception):
    def __init__(self, text):
        self.txt = text

In [6]:
class Foil():
    def __init__(self, name):
        pass
    def get_geometry(self):
        pass
    def get_filename(self):
        pass
    

In [13]:
class Solver():
    def __init__(self, performance_file_name, params_file_name, settings_file_name, foils_file_names, m):
        df = pd.read_csv(performance_file_name, header=None)
        self.performance = dict(zip(df[0], df[1]))
        self.preprocess_performance()

        df = pd.read_csv(settings_file_name, header=None)
        self.settings = dict(zip(df[0], df[1]))

        self.params=pd.read_csv(params_file_name, index_col=False).T
        
        self.foils = []
        for name in foils_file_names:
            self.foils(Foil(name))

        self.geometry = {}
        self.aero = {"CL_cr" : 0, "K_cr" : 0}
        self.tom = m
        

    def XFoil_command_CL(self): #(V, AR, Sw, m, it = 1000, M = 0.0, work_path = path, foil_name = foil):
        command_file=open(work_path + 'commands.in','w')
        command_file.write('load ' + work_path + foil_name + '.txt'+'\n'\
        + foil_name + '\n\
        panel\n\
        oper\n\
        visc ' + str(Re(V, ba(Sw, AR))) + '\n\
        M ' + str(M) + '\n\
        type 1\n\
        vpar\n\
        n ' + str(ncr) + '\n\
        \n\
        iter\n' + str(it) + '\n\
        pacc\n'\
        + work_path + 'polar.dat\n\
        \n\
        cl ' + str(CL_required(V, Sw, m)) +'\n\
        \n\
        \n\
        quit\n')
        command_file.close()
        return 0

    def XFoil_run(file_path = path, xfoil_path = "D:\Desktop\XFoil\\"):
        sep=os.path.sep
        run_xfoil_command = xfoil_path + 'xfoil < ' + file_path + \
        'commands.in'
        os.system(run_xfoil_command)
        return 0
    
    def XFoil_read(file_path = path, data_path = path):
        aero_data_file = open(file_path + 'polar.dat', 'r')
        lines = aero_data_file.readlines()
        aero_data_file.close()
        #delete Xfoil output file ready for next Xfoil run
        os.system('del ' + file_path + 'polar.dat')
        #Linux/OSX: os.system('rm -f ' + file_path + 'polar.dat')
        alpha = float(lines[-1][2: 8])
        cl = float(lines[-1][11: 17])
        cd = float(lines[-1][20: 27])
        return [alpha, cl, cd]
        
    def K_V_solver(V, AR, Sw, m):
        polar = np.array([])
        for v in V:
            XFoil_command_CL(v, AR, Sw, m)
            XFoil_run()
            polar = np.append(polar, [[XFoil_read()]])
            №print(v)
        polar = polar.reshape(polar.size // 3, 3)
        return polar

    
    def AR_selection(self):
        pass

    #FIXME: не дописаны critical_params
    #(обозначения тут правильные)
    def preprocess_settings(self):
        if not self.settings['g']:
            self.settings['g'] = 9.81
            
        if not self.performance['wire_scale_coef']:
            self.performance['wire_scale_coef'] = 1.3
            
        if not self.settings['Density']:
            self.settings['Density'] = 1.22
        
        critical_params = ['TOM', 'take_off_speed', 'CL_take_off', 
                           'eta_prop', 'cruise_speed', 'A_aft', 
                           'B_keel', 'l_stab', 'wire_scale_coef']
        for param in critical_params:
            if isNaN(self.performance.get(param, np.nan)):
                raise InsufficientInputData(f'obligatory parametr {name} not found')

    #FIXME
    #подставить аргументы в функции из aerolib
    def calculate(self):        
        self.geometry["wing_area"] = wing_area()
        self.CL_cr = CL_cruise()
        self.AR, self.K_cr = AR_selection()
        self.geometry["wingspan"] = wingspan()
        self.geometry["aft_area"] = aft_area()
        self.geometry["keel_area"] = keel_area()
        self.geometry["Vdihedral"] = gamma()
        self.geometry["Vtail_area"] = stab_area()
        self.geometry["Pcruise"] = P_cruise()
        self.geometry["wirelength"] = wire_length()
        
    def update_m(self, new_m):
        self.m = new_m
    
    def write_geometry(self):
        with open("geometry.csv", 'w') as f:
            for key in self.geomatry.keys():
                f.write("%s, %s\n" % (key, self.geomatry[key]))

    def get_data_to_weigh():
        return [self.params, self.geometry]

In [14]:
'''  
Ожидаемый формат params-csv-файла
name  index  value conjugate name
0   m1    1.0      1            NaN
1   n1    1.0     10            NaN
2   m2    2.0      2            NaN
3   n2    2.0    100            NaN
4   m3    NaN      3            NaN
5    m    NaN   1000             cy

Ожидаемый формат geomerty-csv-файла
NAME      VALUE
wing_area 1
ba        1
wingspan  1
AR        1
'''

#m (масса ЛА) -- есть сумма
#слагаемые бывают трёх типов:
#1) произведение величин из params с одинаковым индексом
#2) произведение двух величин -- одна из params, одна из geometry (её имя - conjugate name)
#3) величина из params
def weigh(params_file_name, geometry_file_name):
    geom_df = pd.read_csv(geometry_file_name, header=None).loc[1:] #удаление заголовка из DF
    geom = dict(zip(geom_df[0], geom_df[1])) #создание словаря {Cy: <val>, ...}

    #создание таблицы с колонками вида 
    #[<colomn_num>, <name>, <index>, <value>, <conjurate name>]
    params=pd.read_csv(params_file_name, index_col=False).T 

    #создание словаря 
    #{<значение колонки index из params>: <список значений всех параметров с этим индексом>}
    multiply_dict = dict()
    m = 0
    
    for var in params: #итерация по колонкам; в var попадает номер колонки (colomn_num)
        if isNaN(params[var]["value"]):
            raise InsufficientInputData(f"no value for variable '{params[var]['name']}'")

        if not isNaN(params[var]["index"]): #если тип (1)
            #создать в multiply_dict ключ с найденым индексом,
            #со значением пустого списка, если такого ключа ещё нет
            multiply_dict[params[var]["index"]] = multiply_dict.get(params[var]["index"], [])
            #добавить значение этого параметра
            #в список индекса найденного индекса
            multiply_dict[params[var]["index"]].append(params[var]["value"])
        
        elif not isNaN(params[var]["conjugate name"]): #если тип (2)
            if not isNaN(geom.get(params[var]["conjugate name"], np.nan)):  
                m += geom[params[var]["conjugate name"]] * params[var]["value"]
            else:
                raise InsufficientInputData(f"for variable '{params[var]['name']}', '{params[var]['conjugate name']}' "+\
                                            f"is specified as conjugate, but it is no '{params[var]['conjugate name']}'"+\
                                            f" in the performance-file")
                
        elif not isNaN(params[var]["value"]): #если тип (3)
            m += params[var]["value"]

    #вычисление слагаемых типа (1) -- перемножение значения параметров с одинаковыми индексами
    for vals in multiply_dict.values():
        m += math.prod(vals)
    return m


In [18]:
#принимает необходимые даные и делает вывод, успешен ли расчёт 
#опирается на разницу m_target - m_получившаяся
#FIXME: написать
def is_residual(...): 
    pass

In [20]:
# Модуль расчёта геометрических характеристик потребляет на вход: 
# dataframe PERFORMANCE с потребными эксплуатационными характеристиками
# dataframe PARAMS с параметрами электронных компонент, конструкционных материалов, аккумуляторов и т.д.
# пути до файлов селиговского формата с профилем крыла wing_foil и с профилем оперения aft_foil.
# TOM - значение взлётной массы в начальном приближении.

# PERFORMANCE включает параметры take_off_speed, cruise_speed, flight_time, payload

# PARAMS включает параметры m_FPV, m_powerplant, m_flight_control, m_fus, m_servo1, m_servo2,
# line_dens_wire, line_dens_tube1, line_dens_tube2, line_dens_tube3,
# area_dens_LWPLA
# energy_dens_bat
# number_servo1, number_servo2
# l_stab

# Все единицы в СИ
def geom_solver(PERFORMANCE, PARAMS, wing_foil, aft_foil, TOM):
    performance_file_name = input("имя файла performances: ")
    params_file_name = input("имя файла params: ")
    settings_file_name = input("имя файла settings: ") 
    foils_file_names = input("имена файлов с профилями (через запятую): ")
    m = input("масса перевозимого груза: ")
    max_iter = input("максимальное число итераций: ")
    freq_geom_saveing = input("частота сохранения промежуточных геометрии: ")
    
    solver = Solver(performance_file_name, params_file_name, settings_file_name, foils_file_names, m)
    i = 0
    while True:
        solver.calculate()
        weigh(solver.get_data_to_weigh())
        #FIXME: после написания is_residual(...) дописать нужные аргументы и значение new_m
        if not is_residual(...):
            new_m = ...
            solver.update_m(new_m)
        else:
            print(f"сошлось на итерации: {i}")
            solver.write_geometry()
            print("информация сохранена в файл с геометрией")
            break
        
        if i == max_iter:
            print(f"прошло {i} итераций, но расчёт всё ещё не завершён.")
            flag = 2
            while flag not in ['0', '1']:
                flag = input(f"введите 1, чтобы произвести ещё {max_iter} операций, иначе 0: ")
            if flag:
                i = 0
            else:
                solver.write_geometry()
                print("последняя итерация геометрии сохранена")
                break
                
        if i%freq_geom_saveing == 0:
            solver.write_geometry()
            
        i+=1
    
    
    

In [11]:
path = "D:\Desktop\XFoil\SG_6041\\"
foil = "SG6041"

def XFoil_command_CL(V, AR, Sw, m, it = 1000, M = 0.0, work_path = path, foil_name = foil):
    command_file=open(work_path + 'commands.in','w')
    command_file.write('load ' + work_path + foil_name + '.txt'+'\n'\
    + foil_name + '\n\
    panel\n\
    oper\n\
    visc ' + str(Re(V, ba(Sw, AR))) + '\n\
    M ' + str(M) + '\n\
    type 1\n\
    vpar\n\
    n ' + str(ncr) + '\n\
    \n\
    iter\n' + str(it) + '\n\
    pacc\n'\
    + work_path + 'polar.dat\n\
    \n\
    cl ' + str(CL_required(V, Sw, m)) +'\n\
    \n\
    \n\
    quit\n')
    command_file.close()
    return 0

def XFoil_run(file_path = path, xfoil_path = "D:\Desktop\XFoil\\"):
    sep=os.path.sep
    run_xfoil_command = xfoil_path + 'xfoil < ' + file_path + \
    'commands.in'
    os.system(run_xfoil_command)
    return 0
    
def XFoil_read(file_path = path, data_path = path):
    aero_data_file = open(file_path + 'polar.dat', 'r')
    lines = aero_data_file.readlines()
    aero_data_file.close()
    #delete Xfoil output file ready for next Xfoil run
    os.system('del ' + file_path + 'polar.dat')
    #Linux/OSX: os.system('rm -f ' + file_path + 'polar.dat')
    alpha = float(lines[-1][2: 8])
    cl = float(lines[-1][11: 17])
    cd = float(lines[-1][20: 27])
    return [alpha, cl, cd]
    
def K_V_solver(V, AR, Sw, m):
    polar = np.array([])
    for v in V:
        XFoil_command_CL(v, AR, Sw, m)
        XFoil_run()
        polar = np.append(polar, [[XFoil_read()]])
        print(v)
    polar = polar.reshape(polar.size // 3, 3)
    return polar